In [1]:
import csv
import ast
import numpy as np
import time
import random

start = time.time()

# --- Load Billboard Data (BB) ---
BB = []
with open("BB_NYC.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        BB.append({
            'idx': int(row['B_index']),
            'cost': float(row['Cost'])
        })

# --- Load User Influence Data (ub) ---
ub = []
with open("new_ub.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['Influenced Billboards'] = ast.literal_eval(row['Influenced Billboards'])
        ub.append(row)

# --- Load Demand and Budget (from advertiser_1.csv) ---
Dem = []
Budget = []
with open("advertiser_1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Dem.append(float(row['Demand']))
        Budget.append(float(row['Payment']))

print("Budgets:", Budget)

# --- BillboardSlot Class ---
class BillboardSlot:
    def __init__(self, idx):
        self.idx = idx

    def __repr__(self):
        return f"Slot({self.idx})"

# --- Convert BB to BillboardSlot Objects ---
BS = [BillboardSlot(item['idx']) for item in BB[:5000]]


Budgets: [80.0, 120.0, 160.0, 80.0, 80.0, 160.0, 160.0, 80.0, 80.0, 120.0, 120.0, 120.0, 40.0, 80.0, 120.0, 160.0, 80.0, 80.0, 160.0, 120.0, 120.0, 80.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 40.0, 120.0, 80.0, 80.0, 80.0, 120.0, 120.0, 80.0, 160.0, 80.0, 120.0, 80.0, 120.0, 80.0, 40.0, 120.0, 120.0, 120.0, 80.0, 160.0, 80.0, 160.0, 80.0, 160.0, 120.0, 80.0, 80.0, 160.0, 80.0, 120.0, 80.0, 120.0, 120.0, 80.0, 120.0, 80.0, 80.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 80.0, 120.0, 120.0, 120.0, 120.0, 160.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 160.0, 160.0, 80.0, 160.0, 120.0, 80.0, 80.0, 120.0]


In [2]:
# Influence function
def influence(S):
    influence_score = 0.0
    slot_ids = {s.idx if isinstance(s, BillboardSlot) else s for s in S}
   
    for row in ub:
        influenced_billboards = row.get('Influenced Billboards', {})
       
        intersection_result = {int(key): value for key, value in influenced_billboards.items() if int(key) in slot_ids}
       
        product = 1.0
        for value in intersection_result.values():
            product *= (1 - value)
        influence_score += (1 - product)
    return influence_score

In [3]:
def random_pick(BS, Dem, Budget):
    import random

    # Build a mapping from slot.idx to cost
    slot_cost_map = {slot['idx']: slot['cost'] for slot in BB}

    available_slots = {slot.idx for slot in BS}
    used_slots = set()
    Si_list = [set() for _ in Dem]     # Slot sets per product
    total_costs = [0.0 for _ in Dem]   # Total cost per product

    for j in range(len(Dem)):
        demand = Dem[j]
        remaining_budget = Budget[j]
        current_influence = 0.0

        while current_influence < demand:
            remaining = list(available_slots - used_slots - Si_list[j])
            if not remaining:
                print(f"No more slots available to fulfill demand for product {j}.")
                break

            selected = random.choice(remaining)
            selected_cost = slot_cost_map.get(selected, 0)

            if selected_cost > remaining_budget:
                continue  # Skip if this slot exceeds remaining budget

            # Accept the slot
            Si_list[j].add(selected)
            total_costs[j] += selected_cost
            remaining_budget -= selected_cost
            current_influence = influence(Si_list[j])
            used_slots.add(selected)

        if current_influence < demand:
            print(f"Warning: Demand for product {j} not fully satisfied (Influence = {current_influence:.2f}, Needed = {demand:.2f}).")

    return Si_list


In [7]:
# Run the allocation
result = random_pick(BS, Dem, Budget)
total_influence = 0.0

# Print per-product results with Demand
for i, slots in enumerate(result):
    slot_cost = sum(BB[slot]['cost'] for slot in slots if slot < len(BB))
    achieved_influence = influence(slots)
    total_influence += achieved_influence
    print(f"Product {i+1}: Demand = {Dem[i]:.2f}, Influence = {achieved_influence:.2f}, "
          f"Cost = {slot_cost:.2f}, Budget = {Budget[i]:.2f}, Slots = {sorted(list(slots))}")

# Total unique slots and influence
all_selected = set().union(*result)
print("\nTotal Unique Slots Selected:", len(all_selected))
print("Total Combined Influence:",total_influence)
print("Execution Time:", round(time.time() - start, 2), "seconds")

# Count satisfied products
satisfied_count = sum(1 for j in range(len(Dem)) if influence(result[j]) >= Dem[j])
print("Number of products whose demands are satisfied:", satisfied_count, "out of", len(Dem))


Product 1: Demand = 3.00, Influence = 3.10, Cost = 0.14, Budget = 80.00, Slots = [1377, 1515, 1592, 1962, 2410, 2695, 2786, 2826, 2879, 2956, 3096, 3334, 3469, 3502, 3808, 3959, 4192, 4240, 4343, 4524, 4985]
Product 2: Demand = 4.00, Influence = 12.01, Cost = 0.34, Budget = 120.00, Slots = [22, 300, 485, 532, 682, 703, 806, 977, 1290, 1468, 1775, 2434, 2598, 2727, 2947, 2953, 3024, 3073, 3179, 3351, 3859, 4101, 4199, 4374, 4423, 4922]
Product 3: Demand = 4.00, Influence = 4.41, Cost = 0.35, Budget = 160.00, Slots = [156, 157, 1752, 1815, 2359, 2885, 3466, 3709, 3809, 3966]
Product 4: Demand = 3.00, Influence = 9.62, Cost = 0.28, Budget = 80.00, Slots = [563, 1190, 1434, 1660, 1704, 1782, 1841, 2512, 2514, 2790, 2805, 2858, 3171, 3443, 3449, 3801, 3987, 4018, 4729]
Product 5: Demand = 3.00, Influence = 3.95, Cost = 0.24, Budget = 80.00, Slots = [64, 516, 534, 928, 969, 979, 1047, 1212, 1295, 1487, 1491, 1552, 1643, 1724, 1808, 2151, 2418, 2488, 2555, 2739, 2779, 2848, 3120, 3306, 3441, 

In [5]:
h = len(Dem)
count = 0

for j in range(h):
    if influence(result[j]) >= Dem[j]:
        count += 1

print("Number of products whose demands are satisfied:", count, "out of", h)


Number of products whose demands are satisfied: 100 out of 100
